In [1]:
"""Import the required modules"""
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from scipy.linalg import pinv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import timeit

Analyzing welding data by welding types

In [2]:
df2 = pd.read_csv('welding_data_by_welding_types.csv')
df2 = pd.get_dummies(df2).drop('welding_type_GMAW', axis=1)
df2 = df2[['current', 'voltage', 'welding_speed', 'plate_thickness',	'welding_type_SAW',	'welding_type_SMAW',	              'welding_type_GTAW', 'flexural_strength', 'hardness_value', 'residual_stress',
           'tensile_strength']]
df2.head()

,current,voltage,welding_speed,plate_thickness,welding_type_SAW,welding_type_SMAW,welding_type_GTAW,flexural_strength,hardness_value,residual_stress,tensile_strength
0,160,32,120,6,0,0,1,236.2,102.5,152.0,432.6
1,130,32,90,8,0,1,0,270.2,96.4,270.2,332.8
2,100,28,120,8,0,0,1,360.8,86.4,90.5,382.4
3,160,28,90,10,0,0,1,230.0,105.0,165.4,422.8
4,100,28,120,8,1,0,0,276.5,128.4,186.6,385.5


Exploratory data analysis

In [3]:
# pos = 1
# for i in range(4):
#   print(f'\t{df2.columns[i].capitalize()} vs Response values')
#   plt.figure()
#   for j in range(8, 11):
#     plt.subplot(1, 3, pos)
#     plt.scatter(df2.iloc[:, i], df2.iloc[:, j])
#     plt.ylabel(df2.columns[j])
#     plt.xlabel(df2.columns[i])
#     pos += 1
#   plt.tight_layout()
#   plt.show()
#   print('\n')
#   pos = 1

Generate the necessary variables for modeling

In [4]:
X1 = df2.iloc[:, :7]
X = X1.to_numpy()
y1 = df2.iloc[:, -4:]
y = y1.to_numpy()

In [5]:
def relu(x):
   return np.maximum(x, 0, x)

In [6]:
def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + biases
    H = relu(G)
    return H

In [7]:
def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

In [8]:
def MAD(y, y_pred):
    '''Calculates the Mean Absolute Deviation of two arrays of numbers
    Input: y(array), y_pred(array)
    Output: float
    '''
    import numpy as np

    return np.average(np.abs(y - y_pred))

In [9]:
def MAPE(y, y_pred):
    '''
    Calculates the relationship between the average prediction error
    and the average expected target error for several consecutive periods.

    Input: y(array), y_pred(array)
    Output: float
    '''
    import numpy as np

    # Calculate prediction error
    error = y - y_pred

    return np.average((np.abs(error)/y) * 100)

In [10]:
def TS(y, y_pred):
    '''
    This index is used to track the accuracy of the model

    Input: y(array), y_pred(array)
    Output: float
    '''
    import numpy as np

    def MAD(y, y_pred):
        '''Calculates the Mean Absolute Deviation of two arrays of numbers
        Input: y(array), y_pred(array)
        Output: float
        '''
        return np.average(np.abs(y - y_pred))

    return np.sum(y - y_pred) / MAD(y, y_pred)

In [11]:
# Split data in readiness for training.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)

# Convert test data to dataframe and print it.
y_test_df = pd.DataFrame(y_test, columns=['flexural_strength',	'hardness_value',	'residual_stress',	'tensile_strength'])

Predicting individual response separately to optimize prediction

In [13]:
metrics = []
for hidden_size in range(1000, 6500, 1000):
    # print(f'\nELM Analysis for {hidden_size} Node(s)')
    for i in range(y.shape[1]):
        metric = [y1.columns[i], int(hidden_size/200)]
        cols = y[:, i]
        X_train, X_test, y_train, y_test = train_test_split(X, cols,random_state=1, test_size=0.2)

        tic = timeit.timeit() #Start timer
        input_size = X_train.shape[1]
        input_weights = np.random.normal(size=[input_size,hidden_size])
        biases = np.random.normal(size=[hidden_size])
        output_weights = np.dot(pinv2(hidden_nodes(X_train)), y_train)
        prediction = predict(X_test)
        toc = timeit.timeit() #Stop timer

        # Display data
        print(f'\n{y1.columns[i]}: {int(hidden_size/200)} Nodes')
        a = []
        for j in range(len(y_test)):
            a.append([y_test[j], prediction[j]])
        a_df = pd.DataFrame(a, columns=['Expected', 'Predicted'])
        display(a_df)

        # Calculate the necessary metrics
        r2 = r2_score(prediction, y_test)
        mse = mean_squared_error(y_test, prediction)
        rmse = np.sqrt(mse)
        mad = MAD(y_test, prediction)
        mape = MAPE(y_test, prediction)
        ts = TS(y_test, prediction)
        time = abs(toc - tic)

        metric.extend([r2, mse, rmse, mad, mape, ts, time])   # Update metric
        metrics.append(metric)  # Undate metrics


metrics_df = pd.DataFrame(metrics, columns=['Variable', 'Nodes', 'R2', 'MSE', 'RMSE', 'MAD', 'MAPE', 'TS', 'Time'])

display(metrics_df)


flexural_strength: 5 Nodes


,Expected,Predicted
0,218.4,215.735352
1,238.4,231.543031
2,336.8,302.096197
3,230.0,295.654812
4,196.0,194.347920
5,260.0,265.364599
6,345.0,309.861432
7,394.4,356.518980



hardness_value: 5 Nodes


,Expected,Predicted
0,132.5,140.618443
1,116.2,114.319214
2,90.6,78.068746
3,105.0,97.660440
4,142.2,143.689653
5,110.0,115.546214
6,80.5,88.495978
7,65.0,77.354106



residual_stress: 5 Nodes


...


residual_stress: 30 Nodes


,Expected,Predicted
0,188.6,181.930902
1,216.8,220.428586
2,125.8,114.303468
3,165.4,192.262353
4,164.0,170.893519
5,264.6,267.724403
6,108.6,106.752147
7,82.0,87.577242



tensile_strength: 30 Nodes


,Expected,Predicted
0,312.8,316.431745
1,442.6,438.542254
2,396.5,483.027715
3,422.8,372.688306
4,280.2,278.097232
5,304.2,331.490274
6,394.6,418.454299
7,378.5,358.245563


,Variable,Nodes,R2,MSE,RMSE,MAD,MAPE,TS,Time
0,flexural_strength,5,0.604592,1033.778191,32.152421,23.739562,8.205236,2.016789,0.002813
1,hardness_value,5,0.888408,66.235890,8.138543,7.156999,7.949496,-1.921587,0.000208
2,residual_stress,5,0.921690,268.576341,16.388299,10.326147,6.603079,-4.536184,0.003893
3,tensile_strength,5,0.507158,2502.520940,50.025203,36.475915,9.719409,-4.904011,0.006697
4,flexural_strength,10,0.326779,1511.347014,38.876047,29.903672,10.338659,2.175375,0.000893
5,hardness_value,10,0.691672,121.709273,11.032193,7.477440,9.017726,-0.809972,0.011069
6,residual_stress,10,0.952640,159.873438,12.644107,10.125956,7.115696,-2.041876,0.000961
7,tensile_strength,10,0.664152,1424.070711,37.736861,28.449760,7.449570,-3.132928,0.001145
8,flexural_strength,15,0.389169,1355.588788,36.818321,28.918224,10.148071,0.370429,0.018514
9,hardness_value,15,0.750932,104.923397,10.243212,7.474961,9.298026,-2.261998,0.001295
